In [1]:
import pandas as pd
import numpy as np
import pyodbc as db
import warnings
from numpy.random import seed
from numpy.random import randn
from numpy.random import normal
from scipy.stats import ttest_ind
warnings.filterwarnings('ignore')

In [7]:
cnx: db.Connection = db.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")

In [3]:
def get_user_history(TOTOID, UsersList):
    user_bets = f"""
    SELECT u.PartnerUserId,
           o.StakeAmount AS BetAmount,
    	   o.CalculationDate_DT AS Date
    FROM VIEW_sport_OrdersBetsStakes_TotogamingAm AS o
        INNER JOIN VIEW_sport_PartnerUser_TotogamingAm AS u ON u.UserID = o.UserID
    WHERE u.PartnerUserId = {TOTOID}
      AND o.OrderStateID IN (2,3,5,6,8,9,10)
      AND o.CalculationDate_DT >= '2022-07-01'
      AND o.CalculationDate_DT  < CAST(GETDATE() as date)
      AND u.isDeleted NOT IN (1)
      AND o.DeviceTypeID NOT IN (1)

    UNION ALL

    SELECT u.PartnerUserId,
           (CASE WHEN g.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
            WHEN g.GameProviderID NOT IN (48, 10) THEN o.OrderAmount ELSE 0 END) BetAmount,
    	   o.CalculationDate_DT
    FROM casino.orders AS o
    INNER JOIN C_Game AS g on g.GameID=o.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u on u.UserID = o.UserID
    WHERE u.PartnerUserId = {TOTOID}
      AND o.CalculationDate_DT >= '2022-07-01'
      AND o.CalculationDate_DT  < CAST(GETDATE() as date)
      AND o.OperationTypeID IN (3,299)
      AND g.GameProviderID<>3
      AND o.OrderStateID IN (2,3,5,6,8,9,10) AND CASE WHEN g.GameProviderID IN (48, 10) THEN o.TypeId ELSE 0 END IN (0, 1, 5, 8, 18, 33)
    """
    df = pd.read_sql(user_bets, cnx)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['PartnerUserId'] = df['PartnerUserId'].astype(int)
    anchor_date = UsersList[UsersList['PartnerUserId'] == TOTOID].iloc[0,0]
    bet_amounts_after = df[df['Date'] >= anchor_date]['BetAmount'].values
    bet_amounts_before = df[df['Date'] < anchor_date]['BetAmount'].values
    t_stat, p_value = ttest_ind(bet_amounts_before, bet_amounts_after, equal_var=True)
    return t_stat, p_value

In [47]:
for i in [100677520,100741725,101098947,100381567,100401575,100393944,101027480,100453862,100596928,100929633,100739883]:
    print(get_user_history(i, users))

        PartnerUserId    BetAmount       Date
0           100677520  5522.000000 2022-09-28
1           100677520  5000.000000 2022-09-29
2           100677520  5000.000000 2022-09-29
3           100677520  5000.000000 2022-09-29
4           100677520  4518.105882 2022-09-25
...               ...          ...        ...
272170      100677520     0.000000 2023-03-05
272171      100677520     0.000000 2023-03-05
272172      100677520     0.000000 2023-03-05
272173      100677520     0.000000 2023-03-05
272174      100677520    10.000000 2023-03-05

[272175 rows x 3 columns]
2022-10-01 00:00:00
[1358.26771654 3641.73228346 9632.         ...    0.            0.
   10.        ]
[5522. 5000. 5000. ...   50.   50.   50.]
(        PartnerUserId    BetAmount       Date
0           100677520  5522.000000 2022-09-28
1           100677520  5000.000000 2022-09-29
2           100677520  5000.000000 2022-09-29
3           100677520  5000.000000 2022-09-29
4           100677520  4518.105882 2022-09-25

In [4]:
users = pd.read_excel('Z:\\Analytics\\High Values Change Analysis\\High Users.xlsx')
users['Given High Status-Date'] = pd.to_datetime(users['Given High Status-Date'], format='%Y-%m-%d')

In [8]:
t_stats_ = []
p_values_ = []
for i in users['PartnerUserId'].values:
    t_stat, p_value = get_user_history(i, users)
    p_value = "{:.8f}".format(p_value)
    t_stat  = "{:.8f}".format(t_stat)
    print(t_stat, p_value)
    t_stats_.append(t_stat)
    p_values_.append(p_value)

66.98358755 0.00000000
31.10425494 0.00000000
-118.13974655 0.00000000
32.41665839 0.00000000
13.97880099 0.00000000
146.56857937 0.00000000
-8.65315274 0.00000000
19.66038245 0.00000000
26.77189789 0.00000000
27.28013529 0.00000000
0.15138593 0.87967138
62.56286011 0.00000000
7.54453012 0.00000000
3.36641080 0.00076230
-46.20639273 0.00000000
65.31297906 0.00000000
-5.99821986 0.00000000
8.75795953 0.00000000
-35.18024926 0.00000000
75.74548110 0.00000000
10.20525345 0.00000000
23.70410327 0.00000000
146.33577258 0.00000000
154.02419374 0.00000000
133.32752035 0.00000000
-0.82305781 0.41047581
54.99570714 0.00000000
44.37125182 0.00000000
55.69944557 0.00000000
22.77283926 0.00000000
6.72876703 0.00000000
31.10795743 0.00000000
-8.07887319 0.00000000
-16.40659413 0.00000000
-2.99106191 0.00278035
-37.67693524 0.00000000
-1.72178339 0.08511241
62.89353334 0.00000000
109.00010079 0.00000000
0.19077904 0.84869876
18.93532358 0.00000000
7.05607306 0.00000000
-126.66623595 0.00000000
12.56

In [9]:
users['t_stat'] = t_stats_
users['p_value'] = p_values_

In [10]:
users.t_stat.value_counts()

nan              6
66.98358755      1
-4.96381338      1
27.44087156      1
-33.64537607     1
                ..
-299.99686888    1
2.21674488       1
51.58553145      1
1.46252743       1
0.71533628       1
Name: t_stat, Length: 632, dtype: int64

In [56]:
len(t_stats)

102

In [11]:
users.to_excel('t_test.xlsx')

In [4]:
def get_deposit_history(TOTOID, UsersList):
    user_bets = f"""
    DECLARE @StartDate DATE;
    DECLARE @EndDate   DATE;
    SET @StartDate = '2022-07-01'
    SET @EndDate   = CAST(GETDATE() AS DATE)

    SELECT u.PartnerUserId,
    CAST(p.modify_date AS DATE)        AS Date,
    p.Amount                           AS Amount
    FROM Payment AS p
        INNER JOIN  C_PaymentSystem AS ps
            ON ps.PaymentSystemId = p.PaymentSystemID
        INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm AS u
            ON u.UserID = p.UserID
    WHERE u.PartnerUserId = {TOTOID}
      AND p.PaymentTypeID = 2
      AND p.PaymentStatusID = 8
      AND CAST(p.modify_date AS DATE)  < @EndDate
      AND CAST(p.modify_date AS DATE) >= @StartDate
      AND ps.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
      AND u.PartnerID IN (237)
      AND p.SourceID = 2


    """
    df = pd.read_sql(user_bets, cnx)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['PartnerUserId'] = df['PartnerUserId'].astype(int)
    anchor_date = UsersList[UsersList['PartnerUserId'] == TOTOID].iloc[0,0]
    deposits_after = df[df['Date'] >= anchor_date]['Amount'].values
    deposits_before = df[df['Date'] < anchor_date]['Amount'].values
    t_stat, p_value = ttest_ind(deposits_before, deposits_after, equal_var=True)
    return t_stat, p_value

In [5]:
t_stats_dep = []
p_values_dep = []
for i in users['PartnerUserId'].values:
    t_stat, p_value = get_deposit_history(i, users)
    p_value = "{:.8f}".format(p_value)
    t_stat  = "{:.8f}".format(t_stat)
    print('Loop')
    print(t_stat, p_value)
    t_stats_dep.append(t_stat)
    p_values_dep.append(p_value)

KeyboardInterrupt: 

In [12]:
users_arr = []
t_stat_arr = []
p_value_arr =  []
for id in users.PartnerUserId:
    df_user = df[df['PartnerUserId'] == id]
    user_sample = users[users['PartnerUserId'] == id]
    anchor_date = user_sample['Given High Status-Date'].values[0]
    bet_amounts_after = df_user[df_user['Date'] >= anchor_date]['BetAmount'].values
    bet_amounts_before = df_user[df_user['Date'] < anchor_date]['BetAmount'].values
    bet_amounts_after, bet_amounts_before = remove_outliers(bet_amounts_after), remove_outliers(bet_amounts_before)
    t_stat, p_value = ttest_ind(bet_amounts_before, bet_amounts_after,)
    users_arr.append(id)
    t_stat_arr.append(t_stat)
    p_value_arr.append(p_value)

In [13]:
TTest = pd.DataFrame({'PartnerUserId':users_arr, 't-stat':t_stat_arr, 'p-value':p_value_arr})

In [16]:
TTest.isnull().value_counts()

PartnerUserId  t-stat  p-value
False          True    True       461
               False   False      176
dtype: int64

In [ ]:
users

,Given High Status-Date,PartnerUserId,UserName,CasinoID
0,2022-10-01,100677520,arman098040404,496511
1,2022-10-01,100741725,slavikLV83,546264
2,2022-10-01,101098947,ANGEL87g001,926302
3,2022-10-01,100378290,filip_job,137271
4,2022-10-01,100381567,Interkons,154771
...,...,...,...,...
707,2022-10-01,101043500,Hakob1991-,839624
708,2022-10-01,100556431,karlen-84,370321
709,2023-01-05,100609787,artur-h78,425826
710,2023-01-27,100832042,GABHOVO01,247701


In [15]:
anchor_date_arr = []
for user_id in df.PartnerUserId.values:
    date_ = users[users['PartnerUserId'] == int(user_id)].iloc[0, 0]
    anchor_date_arr.append(date_)

In [16]:
df['Anchor'] = anchor_date_arr

In [18]:
df['Before/After'] = df.apply(lambda row: 'After' if row['Date'] >= row['Anchor'] else 'Before', axis=1)

In [19]:
df

,PartnerUserId,Date,BetAmount,GGR,Bets,Platforms,Anchor,Before/After
0,100383244,2023-03-09,5000.0,5000.0,1,Sport,2022-11-01,After
1,100406049,2022-10-14,4200.0,4200.0,1,Sport,2022-10-01,After
2,100696038,2022-11-23,838481.0,146711.0,56,Sport,2022-10-01,After
3,100806223,2022-07-03,12930.0,4504.0,2,Sport,2022-11-01,Before
4,101242586,2023-03-10,2000.0,2000.0,1,Sport,2023-03-17,Before
...,...,...,...,...,...,...,...,...
983907,100423147,2022-09-28,31450.0,-109825.0,263,Casino,2022-11-01,Before
983908,100423147,2022-10-10,189500.0,17400.0,295,Casino,2022-11-01,Before
983909,100423147,2023-02-23,12020.0,-1390.0,6,Casino,2022-11-01,After
983910,100423147,2022-11-25,4450.0,4450.0,29,Casino,2022-11-01,After


In [20]:
bet_amounts_all_before = df[df['Before/After']=='Before']['BetAmount'].values
bet_amounts_all_after = df[df['Before/After']=='After']['BetAmount'].values

t_stat, p_value = ttest_ind(bet_amounts_all_before, bet_amounts_all_after,)
print(t_stat)
print(p_value)

-5.256767505721549
1.4663994223891713e-07


In [21]:
df['Global t_stat'] = t_stat
df['Global_p_value'] = p_value

In [22]:
df.to_excel('Z:\\Analytics\\High Values Change Analysis\\T_Test_Glob.xlsx')

In [23]:
TTest.to_excel('Z:\\Analytics\\High Values Change Analysis\\T_Test.xlsx')

In [1]:
import turtle

# Create a turtle object
t = turtle.Turtle()

# Draw a rectangle
for i in range(2):
    t.forward(100)
    t.right(90)
    t.forward(50)
    t.right(90)

# Close the turtle window
turtle.done()

In [3]:
import turtle

# Create a turtle object
t = turtle.Turtle()

# Set the color and size of the turtle
t.color('red')
t.pensize(5)

# Draw the heart shape
t.left(45)
t.forward(100)
t.circle(50, 180)
t.right(90)
t.circle(50, 180)
t.forward(100)

# Hide the turtle
t.hideturtle()

# Close the turtle window
turtle.done()